<a href="https://colab.research.google.com/github/nateraw/replicate-examples/blob/main/notebooks/replicate_batched_bge_large_en_v1_5_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Running BAAI/bge-large-en-v1.5 on Replicate

In this notebook, we'll see how to run [`BAAI/bge-large-en-v1.5`](https://hf.co/baai/bge-large-en-v1.5) on Replicate - the current SOTA open source model for text embeddings! (as of 10/27/23)

As you'll see, this Replicate model is both better than OpenAI embeddings, and 4x cheaper to run for large scale text embedding.

👀 See the model in the Replicate UI [here](https://replicate.com/nateraw/bge-large-en-v1.5), and more ways to run it (node, curl, docker, etc.) [here](https://replicate.com/nateraw/bge-large-en-v1.5/api).

In [ ]:
%%capture
! pip install replicate

# to count tokens
! pip install transformers sentencepiece

# For our example dataset samsum, we need these
! pip install datasets py7zr scikit-learn

Authenticate with [Replicate](https://replicate.com) :)

In [ ]:
import os
from getpass import getpass
os.environ["REPLICATE_API_TOKEN"] = getpass("Enter your Replicate API Token from:\nhttps://replicate.com/account/api-tokens\n\nPress Enter when done\n")

## From list of text

Quick example from JSON list of text.

Run this to get the model warmed up. Read about how cold boots work on Replicate [here](https://replicate.com/docs/how-does-replicate-work#cold-boots).

In [ ]:
import json

import replicate

texts = [
    "the happy cat",
    "the quick brown fox jumps over the lazy dog",
    "lorem ipsum dolor sit amet",
    "this is a test",
]

output = replicate.run(
    "nateraw/bge-large-en-v1.5:9cf9f015a9cb9c61d1a2610659cdac4a4ca222f2d3707a68517b18c198a9add1",
    input={"texts": json.dumps(texts)}
)

print(output)

[[0.0027728856075555086, 0.04274860396981239, 0.018828075379133224, 0.017545003443956375, -0.047881752252578735, -0.0455041341483593, -0.007684006821364164, 0.057658422738313675, 0.020468177273869514, -0.013831636868417263, 0.03005830943584442, 0.053054098039865494, -0.034432388842105865, -0.006270494777709246, -0.022150667384266853, -0.036757562309503555, -0.004845177289098501, -0.036053892225027084, -0.029423678293824196, 0.028531162068247795, 0.016401097178459167, 0.019110916182398796, -0.07003811001777649, -0.02126355469226837, 0.041573796421289444, 0.06512422859668732, -0.008920572698116302, 0.01838802918791771, 0.05934183672070503, 0.01776735484600067, 0.01889902725815773, -0.01653607003390789, 0.053251445293426514, -0.014244352467358112, -0.011234213598072529, -0.002137875184416771, 0.005704359617084265, 0.010107793845236301, 0.043888937681913376, -0.034957412630319595, 0.010388633236289024, -0.038464806973934174, 0.04057839885354042, -0.04729579761624336, -0.0762246772646904, -

# From jsonl file

I recommend to use a file for making predictions if you've got a larger amount of text to embed (>100 embeddings).

Here's a dummy example to show you the best way to do that.

In [ ]:
%%writefile dummy_example.jsonl
{"text": "the happy cat"}
{"text": "the quick brown fox jumps over the lazy dog"}
{"text": "lorem ipsum dolor sit amet"}
{"text": "this is a test"}

Writing dummy_example.jsonl


In [ ]:
output = replicate.run(
    "nateraw/bge-large-en-v1.5:9cf9f015a9cb9c61d1a2610659cdac4a4ca222f2d3707a68517b18c198a9add1",
    input={"path": open("dummy_example.jsonl", "rb")}
)

In [ ]:
len(output)

4

## Real Example - big jsonl file (via `datasets` library)

Here, we'll encode the whole [samsum](https://hf.co/datasets/samsum) dataset. ~14k examples.

In [ ]:
from pathlib import Path

from datasets import load_dataset

dataset_name = "samsum"
text_field = "dialogue"
outfile_name = "samsum_dialogue.jsonl"

ds = load_dataset(dataset_name, split='train')
ds = ds.remove_columns([x for x in ds.column_names if x != text_field])
ds = ds.rename_column(text_field, "text")
texts = ds["text"]
texts[0]

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

"Amanda: I baked  cookies. Do you want some?\r\nJerry: Sure!\r\nAmanda: I'll bring you tomorrow :-)"

Write to jsonl text file!

In [ ]:
ds.to_json(outfile_name)

Creating json from Arrow format:   0%|          | 0/15 [00:00<?, ?ba/s]

8083570

Looks like this!

In [ ]:
! head -n 5 {outfile_name}

{"text":"Amanda: I baked  cookies. Do you want some?\r\nJerry: Sure!\r\nAmanda: I'll bring you tomorrow :-)"}
{"text":"Olivia: Who are you voting for in this election? \r\nOliver: Liberals as always.\r\nOlivia: Me too!!\r\nOliver: Great"}
{"text":"Tim: Hi, what's up?\r\nKim: Bad mood tbh, I was going to do lots of stuff but ended up procrastinating\r\nTim: What did you plan on doing?\r\nKim: Oh you know, uni stuff and unfucking my room\r\nKim: Maybe tomorrow I'll move my ass and do everything\r\nKim: We were going to defrost a fridge so instead of shopping I'll eat some defrosted veggies\r\nTim: For doing stuff I recommend Pomodoro technique where u use breaks for doing chores\r\nTim: It really helps\r\nKim: thanks, maybe I'll do that\r\nTim: I also like using post-its in kaban style"}
{"text":"Edward: Rachel, I think I'm in ove with Bella..\r\nrachel: Dont say anything else..\r\nEdward: What do you mean??\r\nrachel: Open your fu**ing door.. I'm outside"}
{"text":"Sam: hey  overheard r

## Run Predictions

This time, we'll choose to `convert_to_numpy`, which means our response will be a path to a saved `.npy` file instead of embeddings themselves. This is recommended when you want to compute a lot of embeddings at once, like we're doing here.

In [ ]:
import time

start = time.time()
output = replicate.run(
    "nateraw/bge-large-en-v1.5:9cf9f015a9cb9c61d1a2610659cdac4a4ca222f2d3707a68517b18c198a9add1",
    input=dict(
        path=open(outfile_name, "rb"),
        convert_to_numpy=True,
        batch_size=64
    )
)
time_to_embed = time.time() - start
print(f"that took {time_to_embed:.2f} seconds.")

print("output", output)

that took 65.51 seconds.
output https://replicate.delivery/pbxt/ZpzzGcdZf5VbCCgynfufoXww7MtymKITDa0HfAZOOVsvNNJHB/embeddings.npy


## Load the predictions

Since we chose to convert to numpy, we'll load with numpy here.

In [ ]:
import requests
from io import BytesIO

import numpy as np

embeds = np.load(BytesIO(requests.get(output).content))
embeds.shape

(14732, 1024)

## Price vs. OpenAI

## Pricing vs OpenAI

At the time of writing this, OpenAI's Ada v2 model costs $0.0001 / 1K tokens.

```
Model	Usage
Ada v2	$0.0001 / 1K tokens
```

On replicate, you're charged by the second for the hardware you're running on. In this case, we're using A40 (Large) instances, which cost 0.000725/sec.

👀 Read more about Replicate's pricing [here](https://replicate.com/pricing).

Below, we'll compare both

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-large-en-v1.5")

Prepare a benchmark file with 512 tokens in each example (the max seq length of this model).

Our benchmark will have 5,120,000 tokens.

In [ ]:
text = """\
Lorem ipsum dolor sit amet, consectetur adipiscing elit, \
sed do eiusmod tempor a b
""" * 16  # Not long enough, need >= 512 tokens, so multiply by 16

# no truncation (how many input tokens)
print(len(tokenizer.encode(text, truncation=False, add_special_tokens=False)))
# with truncation (just for fun)
print(len(tokenizer.encode(text, truncation=True, add_special_tokens=False)))

512
512


In [ ]:
from datasets import Dataset

ds = Dataset.from_dict({"text": [text] * 10000})

In [ ]:
def count_tokens(ex):
    ex['num_tokens'] = len(tokenizer.encode(ex["text"], truncation=True, add_special_tokens=False))
    return ex

ds = ds.map(count_tokens)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
total_tokens = sum(ds['num_tokens'])
total_tokens

5120000

In [ ]:
outfile_name = "benchmark.jsonl"
ds.to_json(outfile_name)

Creating json from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

13730000

---

**Note - the time of this cell depends on you *not* running into a cold boot, assuming you ran the cells above recently.**

Even if you do run into a cold boot, the result should be the same. The way we time here is lazy, as the real time is the one you see in the replicate dashboard, which should be ~148 seconds.

In [ ]:
import time

start = time.time()
output = replicate.run(
    "nateraw/bge-large-en-v1.5:9cf9f015a9cb9c61d1a2610659cdac4a4ca222f2d3707a68517b18c198a9add1",
    input=dict(
        path=open(outfile_name, "rb"),
        convert_to_numpy=True,
        batch_size=64
    )
)
time_to_embed = time.time() - start
print(f"that took {time_to_embed:.2f} seconds.")

print("output", output)

that took 151.92 seconds.
output https://replicate.delivery/pbxt/VVrkEaiaem3uHCzAqAOmCaewTobbvrmA20QNpJo8tE39VTyRA/embeddings.npy


In [ ]:
openai_cost = 0.0001  # per 1k tokens
openai_price = total_tokens / 1000 * openai_cost
print(f"OpenAI price: ${openai_price:.3f} USD")

OpenAI price: $0.512 USD


In [ ]:
replicate_price = time_to_embed * 0.000725
print(f"Replicate cost: ${replicate_price:.3f}")

Replicate cost: $0.110


I'm seeing in my replicate dashboard 148 seconds on average, so hard coding that here in case number above is incorrect.

In [ ]:
time_to_embed_actual = 148
replicate_price = time_to_embed_actual * 0.000725
print(f"Replicate cost (actual, from dashboard): ${replicate_price:.3f}")

Replicate cost (actual, from dashboard): $0.107
